In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile
import os

# Define the path to the zip file
zip_file_path = r"/content/drive/MyDrive/new image set.zip"
# Define the directory where you want to extract the files
extract_to_path = '/content/sample_data'

# Create the directory if it doesn't exist
os.makedirs(extract_to_path, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print(f'Files extracted to {extract_to_path}')


Files extracted to /content/sample_data


In [11]:
from tensorflow.keras.utils import to_categorical

# Convert labels to one-hot encoded format
y_train = to_categorical(y_train, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)


In [6]:
!pip install gradio



  Using cached gradio-4.37.2-py3-none-any.whl (12.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━

In [7]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import gradio as gr
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

# Define paths
dataset_folder = r"/content/sample_data/Dataset_BUSI_with_GT" # Path to the images folder

# Define image size and class labels
IMG_SIZE = 128  # Image size for the model input
class_names = ['benign', 'malignant', 'normal']
class_mapping = {name: idx for idx, name in enumerate(class_names)}

# Load images from each class folder
def load_images_from_folder(folder, class_label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load in grayscale
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # Resize image
            img = np.expand_dims(img, axis=-1)  # Add channel dimension
            images.append(img)
            labels.append(class_label)
    return images, labels

images = []
labels = []

# Load images from each class folder
for class_name, class_label in class_mapping.items():
    folder_path = os.path.join(dataset_folder, class_name)
    imgs, lbls = load_images_from_folder(folder_path, class_label)
    images.extend(imgs)
    labels.extend(lbls)

images = np.array(images)
labels = np.array(labels)

# Convert labels to one-hot encoded format
y_train = to_categorical(labels, num_classes=3)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(images, y_train, test_size=0.2, random_state=42)

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Create Data Generators for training and testing
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
test_generator = test_datagen.flow(X_test, y_test, batch_size=32)

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),  # Input is grayscale, hence single channel
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Three classes: Benign, Malignant, Normal
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=25
)

# Save the model
model.save('breast_cancer_detection_model3.h5')

# Define the prediction function
def predict_cancer(images):
    results = []
    for img in images:
        # Ensure image is in RGB format and convert to grayscale
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)  # Convert to grayscale
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # Resize to match model input
        img = np.expand_dims(img, axis=-1)  # Add channel dimension
        img = img / 255.0  # Normalize
        img = np.expand_dims(img, axis=0)  # Add batch dimension

        # Make prediction
        prediction = model.predict(img)
        class_idx = np.argmax(prediction[0])
        class_name = class_names[class_idx]
        probability = np.max(prediction[0])
        results.append(f"{class_name} (Probability: {probability:.2f})")

    return results

# Define Gradio interface
input_images = gr.Image(type='numpy', label='Upload Ultrasound Images', tool='editor', source='upload', shape=(IMG_SIZE, IMG_SIZE))
output_labels = gr.Textbox(label='Predictions')

gr_interface = gr.Interface(
    fn=predict_cancer,
    inputs=input_images,
    outputs=output_labels,
    title="Breast Cancer Detection from Ultrasound Images",
    description="Upload multiple breast ultrasound images to get predictions on whether they show benign, malignant, or normal conditions."
)

# Launch the Gradio app
if __name__ == "__main__":
    gr_interface.launch()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 14, 128)       0

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


TypeError: Image.__init__() got an unexpected keyword argument 'tool'

In [13]:
# # Save the model
# model.save('breast_cancer_detection_model2.h5')

In [15]:
!pip install gradio tensorflow opencv-python-headless


In [8]:
# Define the prediction function
def predict_cancer(images):
    results = []
    for img in images:
        # Convert image to grayscale (if it's not already), resize, and normalize
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # Resize to match model input
        img = np.expand_dims(img, axis=-1)  # Add channel dimension
        img = img / 255.0  # Normalize
        img = np.expand_dims(img, axis=0)  # Add batch dimension

        # Make prediction
        prediction = model.predict(img)
        class_idx = np.argmax(prediction[0])
        class_name = class_names[class_idx]
        probability = np.max(prediction[0])
        results.append(f"{class_name} (Probability: {probability:.2f})")

    return results

# Define Gradio interface
def classify_images(images):
    return predict_cancer(images)

# # Update Gradio interface configuration
# input_images = gr.Image(type='numpy', label='Upload Ultrasound Images', source='upload', tool=None)  # Removed `tool` argument and added `source='upload'`
# output_labels = gr.Textbox(label='Predictions')

# Define Gradio interface
input_images = gr.Image(type='numpy', label='Upload Ultrasound Images')
output_labels = gr.Textbox(label='Predictions')

gr_interface = gr.Interface(
    fn=classify_images,
    inputs=input_images,
    outputs=output_labels,
    title="Breast Cancer Detection from Ultrasound Images",
    description="Upload multiple breast ultrasound images to get predictions on whether they show benign, malignant, or normal conditions."
)

# Launch the Gradio app
if __name__ == "__main__":
    gr_interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://fd6a25a058bd67fbe1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [9]:
import gradio as gr
import numpy as np
import cv2  # Ensure you have opencv-python installed
from tensorflow.keras.models import load_model  # Ensure you have TensorFlow installed

# Load your trained model
model = load_model('/content/breast_cancer_detection_model3.h5')  # Update this path to your actual model file

# Define class names according to your model
class_names = ['benign', 'malignant', 'normal']  # Update this list if different

# Define the prediction function
def predict_cancer(images):
    results = []
    for img in images:
        # Convert image to grayscale (if it's not already), resize, and normalize
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)  # Convert to grayscale if not already
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # Resize to match model input
        img = np.expand_dims(img, axis=-1)  # Add channel dimension
        img = img / 255.0  # Normalize
        img = np.expand_dims(img, axis=0)  # Add batch dimension

        # Make prediction
        prediction = model.predict(img)
        class_idx = np.argmax(prediction[0])
        class_name = class_names[class_idx]
        probability = np.max(prediction[0])
        results.append(f"{class_name} (Probability: {probability:.2f})")

    return results

# Define Gradio interface
def classify_images(images):
    if not isinstance(images, list):  # Ensure `images` is a list of images
        images = [images]
    return predict_cancer(images)

# Define the Gradio interface
input_images = gr.Image(type='numpy', label='Upload Ultrasound Images')
output_labels = gr.Textbox(label='Predictions')

gr_interface = gr.Interface(
    fn=classify_images,
    inputs=input_images,
    outputs=output_labels,
    title="Breast Cancer Detection from Ultrasound Images",
    description="Upload multiple breast ultrasound images to get predictions on whether they show benign, malignant, or normal conditions."
)

# Launch the Gradio app
if __name__ == "__main__":
    gr_interface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://134538bfc74bbc1b4e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
